## Import Packages

In [9]:
import sys
import os

import fiona
import geopandas as gpd
import rioxarray
from pathlib import Path
from shapely.geometry import mapping

In [10]:
rel_path = "../src"
directory_path = os.path.abspath(os.path.join(os.getcwd(), rel_path))
sys.path.append(directory_path)

from nex_gddp_cmip6 import get_nex_dataset, TIME_OPTIMIZED_SCENARIOS, AVAILABLE_VARIABLES

## Define Variables

In [2]:
folderpath = '../data'
filename = 'gdf_easternmountain_polygons'
random_state = 42

## Define Functions

In [5]:
def load_gdf(folderpath, filename):
    """
    Load a GeoDataFrame from the specified 'processed' directory.

    Parameters:
    - folderpath (str): The path to the main folder containing the 'processed' subfolder.
    - filename (str): The name of the file (with extension) to load from the 'processed' directory.

    Returns:
    - GeoDataFrame: A GeoDataFrame loaded from the specified file in the 'processed' subfolder.
    """
    # Create a Path object for folderpath to ensure correct path manipulation
    folder = Path(folderpath)

    # Construct the file path for the processed version of the file
    filepath = folder / 'processed' / filename
    
    # Load and return the GeoDataFrame
    return gpd.read_file(str(filepath))

In [7]:
def clip_dataset(dataset, geodataframe):
    """
    Clip a dataset by a GeoDataFrame's boundaries.

    Parameters:
    - dataset: The dataset to be clipped.
    - geodataframe: A GeoDataFrame that defines the region to clip.

    Returns:
    - The clipped dataset.
    """
    rio_dataset = dataset.rio.write_crs("EPSG:4326")
    return rio_dataset.rio.clip(geodataframe.geometry.apply(mapping), geodataframe.crs)

## Execute Functions

In [11]:
# Load datasets from S3
ds_historical = get_nex_dataset(AVAILABLE_VARIABLES,["historical"])
ds_projection = get_nex_dataset(AVAILABLE_VARIABLES,["projection"])

In [15]:
# Load GeoDataFrame
gdf = load_gdf(folderpath, filename)

# Clip datasets
ds_historical_clipped = clip_dataset(ds_historical, gdf)
ds_projection_clipped = clip_dataset(ds_projection, gdf)

In [16]:
ds_historical_clipped

<xarray.Dataset> Size: 8GB
Dimensions:      (lat: 19, lon: 25, model: 20, scenario: 1, time: 23741)
Coordinates:
  * lat          (lat) float64 152B 35.12 35.38 35.62 ... 39.12 39.38 39.62
  * lon          (lon) float64 200B 96.38 96.62 96.88 ... 101.9 102.1 102.4
  * model        (model) <U10 800B 'ACCESS-CM2' 'ACCESS-ESM' ... 'UKESM1-0-L'
  * scenario     (scenario) <U10 40B 'historical'
  * time         (time) datetime64[ns] 190kB 1950-01-01T12:00:00 ... 2014-12-...
    spatial_ref  int32 4B 0
Data variables:
    hurs         (model, scenario, time, lat, lon) float32 902MB dask.array<chunksize=(20, 1, 23741, 10, 5), meta=np.ndarray>
    huss         (model, scenario, time, lat, lon) float32 902MB dask.array<chunksize=(20, 1, 23741, 10, 5), meta=np.ndarray>
    pr           (model, scenario, time, lat, lon) float32 902MB dask.array<chunksize=(20, 1, 23741, 10, 5), meta=np.ndarray>
    rlds         (model, scenario, time, lat, lon) float32 902MB dask.array<chunksize=(20, 1, 23741, 10, 5), meta=np.ndarray>
    rsds         (model, scenario, time, lat, lon) float32 902MB dask.array<chunksize=(20, 1, 23741, 10, 5), meta=np.ndarray>
    sfcWind      (model, scenario, time, lat, lon) float32 902MB dask.array<chunksize=(20, 1, 23741, 10, 5), meta=np.ndarray>
    tas          (model, scenario, time, lat, lon) float32 902MB dask.array<chunksize=(20, 1, 23741, 10, 5), meta=np.ndarray>
    tasmax       (model, scenario, time, lat, lon) float32 902MB dask.array<chunksize=(20, 1, 23741, 10, 5), meta=np.ndarray>
    tasmin       (model, scenario, time, lat, lon) float32 902MB dask.array<chunksize=(20, 1, 23741, 10, 5), meta=np.ndarray>
Attributes: (12/22)
    Conventions:           CF-1.7
    activity:              NEX-GDDP-CMIP6
    cmip6_institution_id:  CSIRO-ARCCSS
    cmip6_license:         CC-BY 4.0
    cmip6_source_id:       ACCESS-CM2
    contact:               Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bridget...
    ...                    ...
    scenario:              historical
    source:                BCSD
    title:                 ACCESS-CM2, r1i1p1f1, historical, global downscale...
    tracking_id:           f9f489de-f5f7-43de-8f55-f0c001fad19a
    variant_label:         r1i1p1f1
    version:               1.1

In [17]:
ds_projection_clipped

<xarray.Dataset> Size: 43GB
Dimensions:      (lat: 19, lon: 25, model: 20, scenario: 4, time: 31411)
Coordinates:
  * lat          (lat) float64 152B 35.12 35.38 35.62 ... 39.12 39.38 39.62
  * lon          (lon) float64 200B 96.38 96.62 96.88 ... 101.9 102.1 102.4
  * model        (model) <U10 800B 'ACCESS-CM2' 'ACCESS-ESM' ... 'UKESM1-0-L'
  * scenario     (scenario) <U6 96B 'ssp126' 'ssp245' 'ssp370' 'ssp585'
  * time         (time) datetime64[ns] 251kB 2015-01-01T12:00:00 ... 2100-12-...
    spatial_ref  int32 4B 0
Data variables:
    hurs         (model, scenario, time, lat, lon) float32 5GB dask.array<chunksize=(8, 4, 31411, 10, 5), meta=np.ndarray>
    huss         (model, scenario, time, lat, lon) float32 5GB dask.array<chunksize=(8, 4, 31411, 10, 5), meta=np.ndarray>
    pr           (model, scenario, time, lat, lon) float32 5GB dask.array<chunksize=(8, 4, 31411, 10, 5), meta=np.ndarray>
    rlds         (model, scenario, time, lat, lon) float32 5GB dask.array<chunksize=(8, 4, 31411, 10, 5), meta=np.ndarray>
    rsds         (model, scenario, time, lat, lon) float32 5GB dask.array<chunksize=(8, 4, 31411, 10, 5), meta=np.ndarray>
    sfcWind      (model, scenario, time, lat, lon) float32 5GB dask.array<chunksize=(8, 4, 31411, 10, 5), meta=np.ndarray>
    tas          (model, scenario, time, lat, lon) float32 5GB dask.array<chunksize=(8, 4, 31411, 10, 5), meta=np.ndarray>
    tasmax       (model, scenario, time, lat, lon) float32 5GB dask.array<chunksize=(8, 4, 31411, 10, 5), meta=np.ndarray>
    tasmin       (model, scenario, time, lat, lon) float32 5GB dask.array<chunksize=(8, 4, 31411, 10, 5), meta=np.ndarray>
Attributes: (12/22)
    Conventions:           CF-1.7
    activity:              NEX-GDDP-CMIP6
    cmip6_institution_id:  CSIRO-ARCCSS
    cmip6_license:         CC-BY 4.0
    cmip6_source_id:       ACCESS-CM2
    contact:               Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bridget...
    ...                    ...
    scenario:              ssp126
    source:                BCSD
    title:                 ACCESS-CM2, r1i1p1f1, ssp126, global downscaled CM...
    tracking_id:           be97dab5-cba9-412b-bb1f-1dd9d2fd8aa5
    variant_label:         r1i1p1f1
    version:               1.1

In [19]:
# Calculate share of null values to confirm that the data is masked
ds_historical_clipped['tasmin'].isnull().sum().values / (len(ds_historical_clipped.lat) * len(ds_historical_clipped.lon) * len(ds_historical_clipped.time) * len(ds_historical_clipped.model))

0.7646286526306921